# Python and OOP
- Inheritance: Attritube lookup (**X.name**)
- Polymorphism: In **X.method**, the meaning of **method** depends on the class of **X**.
- Encapsulation: method and operator implementation

## Polymorphism
Polymorphism in Python is based on object interface  
In C++, you might try to overload methods by their argument

In [1]:
class C(object):
    def method(self, x):
        print(x)
    def method(self, x, y):
        print(x, y)
        
c = C()
c.method(1)

TypeError: method() missing 1 required positional argument: 'y'

However, it doesn't work in Python.  
Only the last definition will be retained.  
There can be only one definition of a method name

## Compostion
Embedding other objects in a container object, and activating them to implement container methods.  

## Delegation
A special form of composition, with a single embedded object managed by a wrapper class that retains most or all the embedded object's interface  
Often implemented with **`__getattr__`**  

In [2]:
class Wrapper(object):
    def __init__(self, obj):
        self.wrapped = obj
    def __getattr__(self, attrname):
        print('Trace: '+attrname)
        # getattr(x, n) works like x.__dict__[n] except that it does an inheritnace search
        return getattr(self.wrapped, attrname)
    
x = Wrapper([1, 2, 3])
x.append(4)

Trace: append


In [3]:
x.wrapped

[1, 2, 3, 4]

## Pseudoprivate Class Attributes (Mangled names)
Names that start with two underscores but don't end with two underscoress (**__X**)  
They're automatatically expanded to include the name of the enclosing class at their front.  (e.g. **`__X`** in **Spam** class-> **`_Spam__X`**)  

In [4]:
class C(object):
    def __print_value(self, x):
        print(x)

c = C()
c.__print_value(1)

AttributeError: 'C' object has no attribute '__print_value'

In [5]:
# These attribute can still be referenced

c._C__print_value(1)

1


- Works for both class attributes (including method names) and instnace attribute names assigned to **self**  
- Mangled names are sometimes misleadingly called "private attributes", but this is just a way to localize a name to the class that created it. It does not prevent access by code outside the class
- Avoid namespace collisions but not to restrict access

- Usage
    - Larger frameworks
    - Method that is intended for use only within a class that may be mixed into other classes (especially usefule in multiple inhertiance)
    
Although it's possible to emulate true access controls in Python classses, this is rarely done in practive, even for large systems

## Methods Are Objects

In [6]:
class Spam(object):
    def doit(self, message):
        print(message)

- Unbound (class) method objects: without self
    - Accessing a function attribute by qualifying the class

In [7]:
obj = Spam()
t = Spam.doit         # a pure function in Python3
t(obj, 'Unbound')

Unbound


- Bound (instance) method object: self + function paris
    - Accessing a function attribute in a class by qualifying an instance

In [8]:
obj = Spam()
x = obj.doit           # instance + function
x('Bounded')  

Bounded


### Unbound Methods in Python3
- In Pytonh3 it' OK to call a method without an instance
- Call it only throught the class and never through an instnace  
- Python3 passes along an instance to methods only for throught-instance calls

In [10]:
class Selfless(object):
    def __init__(self, data):
        self.data = data
    def selfless(arg1, arg2):
        return arg1 + arg2
    def normal(self, arg1, arg2):
        return self.data + arg1 + arg2
    
x = Selfless(2)

x.normal(1, 2)

5

In [9]:
Selfless.selfless(1, 2)

NameError: name 'Selfless' is not defined

Because of this change, the **staticmethod** built-in function and decorator is not needed in Python3 for methods without a **self** argument.

## Generic Object Factories
Sometimes, class-based designs require objects to be created in response to conditions that can't be predicted when a program is written  
Factories can be a major undertaking in a strongly typed language such as C++ but are almost trivial to implement in Python  
Factory might allow code to be insulated from the details of dynamically configured object construction

In [11]:
def factory(aClass, *pargs, **kargs):
    return aClass(*pargs, **kargs)

class Spam(object):
    def doit(self, message):
        print(message)
    
class Person:
    def __init__(self, name, job=None):
        self.name = name
        self.job = job
        
obj1 = factory(Spam)
obj2 = factory(Person, 'Bob')
print(obj1)
print(obj2)

## Multiple Inheritance: "Mix-in" Classes
- Pros: Objects obtain the union of the behavior in all their superclasses
- Cons: It can pose a conflict whne the same method  


- Most commonly used in "mix in " general purpose methods from superclasses  
- Besides the utility, mix-ins optimize code maintenance since only one change needs to be made

### Attribute Searching
Traverses all superclasses in the class header from left to right until a match is found

#### DFLR vs MRO
- In classic classes (DFLR: Depth-Fisrt, Left to Right)
    - Depth first all the way to the top of ineritance tree for all cases
- In new-style classes (MRO: Method Resolution Order)
    - The same as DFLR in normal cases
    - In Dimond Patterns (multiple classes in a tree share a common superclass)
        - Proceed across by tree levels before moving up (more like breadth-first)
        - Designed to visit such a shared superclass just once, and after all its subclasses. 

#### Default vs Explicit
- Default(**self.method()**)
    - Chooses the first occurrence of an attribute
- Explicit(**superclass.method(self)**)
    - Explicitly reference the method in superclass
    - It can break the conflict and overrides the search's default

### Collector Module
To make importing tools even easier, we can provide a collector module that combines them in a single namespace

```Python
# lister.py
from listinstance import ListInstance
from listinherited import ListInherited
from listtree import ListTree

Lister = ListTree
```


---
```Python
# uselister.py
import lister
lister.Lister

```

## Virtual Data
In Python classes, some names associated with instance data may not be stored at the instance itself.  
(e.g. new-style properties, slots, descriptors and attribute computed in tools like **`__getattr__`**)  
None of these "Virtual"  attributes' names are stored in **`__dict__`**

Further information will be mention in the next chapter.